# Binary stars evolutions and binary black holes

Giacomo Menegatti, Dario Puggioni, Laura Schulze, Savina Tsichli

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import regex as re
from scipy.stats import pointbiserialr, spearmanr


## Loading the dataset
The dataset is divided in files depending on the CE efficiency $ \alpha $ and the metallicity $ Z $ of the two stars. All the data are loaded into a pandas dataframe containing also the two parameters.


In [ ]:
alpha = [0.5, 1, 3, 5] # CE efficiency
Z = [2e-4, 4e-4, 8e-4, 1.2e-3, 1.6e-3, 2e-3, 4e-3, 6e-3, 8e-3, 1.2e-2, 1.6e-2, 2e-2]  #Metallicity value

data = [] #Data list 

sim_data = pd.DataFrame({'alpha':[], 'Z':[], 'MtotZAMS':[], 'num_mergers':[]})
#Appending all the values in a single big dataframe

index = 0
for a in alpha:
    for m in Z:
        # The simulation data first row contains the MtotZAMS and the number of merger for each alpha and metallicity 
        df = pd.read_csv(f'stable_MT_vs_CE/A{a}/MTCE_BBHs_{m}.txt', sep=' ', nrows = 1, header=0) 
        sim_data.loc[index] = [a, m, df.iloc[0,0], df.iloc[0,1]]  #Adding the row to the sim_data df
        index = index + 1 
    
        df = pd.read_csv(f'stable_MT_vs_CE/A{a}/MTCE_BBHs_{m}.txt', header=2, sep= ' ')     # Simulation data
        df['alpha'],df['Z'] = a, m                                 #Adding the alpha and Z paramtere in the table
        
        data.append(df)

data = pd.concat(data)
data.columns = [re.sub('col.*:|/.*$', '', name) for name in data.columns]


In [ ]:
data.columns.values[9] = 'kick_1'
data.columns.values[10] = 'kick_2'

print('Data columns: \n ', data.columns.values)
data.reset_index(drop=True,inplace=True)
data


In [ ]:
CE_data = data.query('CE == True')
MT_data = data.query('CE == False')
print(f'CE_data shape={CE_data.shape}, MT_data shape={MT_data.shape}')

print(f'There are {CE_data.shape[0]} BBHs evolving from common envelope, and {MT_data.shape[0]} BBHs evolving from mass transfer')
s_data = data.sample(n=150000, random_state=42)  # A smaller datafreme picking random data, 150k rows above 2.96 millions
s_CE_data = s_data.query('CE == True')
s_MT_data = s_data.query('CE == False')


In [ ]:
#sns.scatterplot(data.query('alpha==0.5 and Z == 2e-4'), x='m1rem/Msun', y='m2rem/Msun', hue = 'CE', markers='.')



In [ ]:
numeric_columns = [col for col in data.columns if col != 'CE' and col != 'ID']

correlation_df = pd.DataFrame(columns=['Feature', 'Spearman_Correlation','Biserial_Correlation'])
correlation_data = []
for column in numeric_columns:
    spearman_corr, _ = spearmanr(data['CE'], data[column])
    biserial_corr, _ = pointbiserialr(data['CE'], data[column])
    correlation_data.append({'Feature':column,'Spearman_Correlation':spearman_corr,'Biserial_Correlation':biserial_corr})
correlation_df = pd.DataFrame(correlation_data)
correlation_df

#I don't know well these two parameters, but the Spearman correlation is used to see how tho continues variables correlate monotonically
#It ranges in the interval [-1,+1], where -1 indicates a perfect decrescent correlation, 0 indicates no correlation and +1 indicates 
#a perfect crescent correlation. But we don't have two continues variables, so I adapted with values 0 and 1. The biserial correlation
#is more specific for our case, but maybe it works only for a correlation of the linear type, but I don't know what does it mean in the binary case,
#so maybe the biserial_correlation is more correct. Anyway these two statisctical indicators give us similar values.


In [ ]:
numeric_columns = [col for col in data.columns if col != 'CE' and col != 'ID']
fig, axs = plt.subplots(nrows=11, ncols=2, figsize=(12, 40))
for i, col in enumerate(numeric_columns):
    ax = axs[i//2, i%2] 
    data[data.CE == True][col].hist(bins=20, color='blue', alpha=0.5, label='CE True', ax=ax)
    data[data.CE == False][col].hist(bins=20, color='orange', alpha=0.5, label='CE False', ax=ax)
    ax.grid(None)
    ax.legend(loc='best')
    ax.set_xlabel(col)
    ax.set_title(f'{col},     biserial correlation = {correlation_df.Biserial_Correlation[i]:.4f}')


plt.tight_layout()
plt.show()

##For example, for column m1ZAMS, the biserial correlation is ~-0.46. In the histogram we see that lower values favor the condition 'True', 
#while higher value favor the condition 'False'. For the column 'v1z' the biserial correlation is ~0.0003, that means no correlation. We can see 
#in the histogram that data are equally distributed for both conditions.
